<a href="https://colab.research.google.com/github/mvish7/PyTorch/blob/master/NN_in_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

necessary imports

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

fun to calculate number of correct preds

In [0]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

creating n/w class and building the fwd method

In [0]:
class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    
    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
    
  def forward(self,t):
    #implementing fwd pass
    
    # input layer
    t=t #returning the tensor as it as, no need of processing
    
    # hidden layer 1 
    t=self.conv1(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    
    #hidden layer 2
    t=self.conv2(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    #hidden layer 3
    t=t.reshape(-1,12*4*4)  #doing the flattening operation before passing the tensor from conv layer to the linear layer
    t=self.fc1(t)
    t=F.relu(t)
    
    #hidden layer 4
    t=self.fc2(t)
    t=F.relu(t)
    
    #output layer
    t=self.out(t)
    #t=F.softmax(t, dim=1)  no need to use softmax here, coz we are using cross entropy as our loss function and it has built in softmax.
    
    
    return t
  
      

data downloaded

In [5]:
trainset=torchvision.datasets.FashionMNIST(root='./data/FashionMNIST',
                                           train=True, 
                                           download=True,
               transform=transforms.Compose([transforms.ToTensor()]))

0it [00:00, ?it/s]

26427392it [00:01, 13676592.61it/s]                              


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 103227.73it/s]           
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:01, 4382432.33it/s]                            
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 31279.69it/s]            

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


downloading the test data now

In [0]:
testset = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', train=False,
                                       download=True, 
                                           transform=transforms.Compose([transforms.ToTensor()]))

In [0]:
network=Network()

creating batch of 100 images, using dataloader class

In [0]:
data_loader=torch.utils.data.DataLoader(trainset,batch_size=100, shuffle=true)
batch=next(iter(data_loader))
images, labels=batch

creating dataloader instance for test data

In [0]:
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False)

calculating the loss

In [0]:
preds= network(images)
loss_fun= nn.CrossEntropyLoss()
#loss.item() #item method gives out the numerical vlaue of the loss

calculating the gradients with backward method

In [0]:
loss.backward()

now using this gradients to update the network weights using optimizer

In [0]:
optimizer=optim.Adam(network.parameters(), lr=0.015)

In [11]:
loss.item() #just checking the loss once again

2.321523427963257

In [12]:
get_num_correct(preds, labels) # we have 12/100 correct

10

now updating the weights using step() method

In [0]:
optimizer.step()

now let's do this steps once again and see of n/w actially learns from the gradients we calculated and updated

In [14]:
preds= network(images)
loss= F.cross_entropy(preds, labels)
loss.item()

2.2917630672454834

In [15]:
get_num_correct(preds, labels)

12

yayy! we got reduced loss and more correct predictions

now creating the training loop

In [17]:
images.shape

torch.Size([100, 1, 28, 28])

setting the training loop

In [44]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        images, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        preds = network(images)
        loss = loss_fun(preds, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.001
[1,   101] loss: 0.115
[1,   201] loss: 0.115
[1,   301] loss: 0.115
[1,   401] loss: 0.115
[1,   501] loss: 0.115
[2,     1] loss: 0.001
[2,   101] loss: 0.115
[2,   201] loss: 0.115
[2,   301] loss: 0.115
[2,   401] loss: 0.115
[2,   501] loss: 0.115
[3,     1] loss: 0.001
[3,   101] loss: 0.115
[3,   201] loss: 0.115
[3,   301] loss: 0.115
[3,   401] loss: 0.115
[3,   501] loss: 0.115
[4,     1] loss: 0.001
[4,   101] loss: 0.115
[4,   201] loss: 0.115
[4,   301] loss: 0.115
[4,   401] loss: 0.115
[4,   501] loss: 0.115
[5,     1] loss: 0.001
[5,   101] loss: 0.115
[5,   201] loss: 0.115
[5,   301] loss: 0.115
[5,   401] loss: 0.115
[5,   501] loss: 0.115
Finished Training


In [45]:
test_imgs,test_label=next(iter(testloader))
preds=network(test_imgs)
get_num_correct(preds,test_label)

5

so 40/50 were predicted correctly

In [46]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        preds = network(images)
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %
